# Import

In [1]:
import os
import sys
module_path = os.path.join("./KeyPoint-Analysis/KPG/")
sys.path.insert(0, module_path)
from rouge_setbase import preprocess_dataset, compute_rouge, compute_rouge_max
from softF1 import softevaluation
root_dir_pth = "../"

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
2025-02-22 03:20:04.489189: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-22 03:20:04.515466: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-22 03:20:04.515494: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-22 03:20:04.516421: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Una

In [2]:
sys.path.insert(1, '../')
from utils.prompting import *
from utils.postprocessing import *
from utils.g_eval import *

In [3]:
import pandas as pd
import re
import ast
import json
import statistics
import openai
from multiprocessing import Pool
openai.api_key = "YOUR API KEY"

# Read Predicted Summary

In [4]:
qqsum_output_path = "../output/atlas-xl-seed2-lgret-lglm"

In [5]:
ground_truth_df = pd.read_pickle("../data/test/test.pkl")
ground_truth_df = ground_truth_df.drop(columns=['reviews', 'reviewText', 'title', 'fact_check_article'])
ground_truth_df['retrieved_relevant_sent_len'] = ground_truth_df['retrieved_relevant_sent'].str.len()

predicted_df = pd.read_json(qqsum_output_path + "/test-result.jsonl", lines=True)
predicted_df['passages'] = predicted_df['passages'].apply(lambda x: [pas['text'] for pas in x])
predicted_df['passages_len'] = predicted_df['passages'].str.len()
predicted_df = predicted_df[['query', 'generation', 'passages', 'passages_scores', 'passages_len', 'comment_clusters', 'id']]
df = ground_truth_df.merge(predicted_df, on=['query', 'id'])

In [6]:
# df = df[df['passages'].str.len() > 0]
# df['passages_scores_min_thres_ref_ground'] = df.apply(lambda row: row['passages_scores'][:len(row['retrieved_relevant_sent'])][-1], axis=1)
# df['passages_scores_min_thres_ref_ground'].mean()

In [7]:
df['summary'] = df['generation'].apply(lambda x: re.findall("\[\/INST\] *((.+\n*)+)$", x)[0][0].replace("</s>", ""))
df['summary'] = df['summary'].apply(lambda x: re.sub("(Therefore|Thus)(.+\n*)+$", "", x))
df['final_summary'] = df['summary'].apply(lambda x: re.findall("(While[^\n]+\n+(\+ *[0-9]+[^\n]+\n*)+)", x))
df['final_summary'] = df['final_summary'].apply(lambda x: [e[0] for e in x])
df['final_summary_text'] = df['final_summary'].apply(lambda x: "\n\n".join(x[:1]))

## Post-process summary into KPs

### Trial

In [8]:
row = df.iloc[0]
print(row['id'])
get_kp_from_summary(row['final_summary_text'])

3609


'```json\n[{"key_point": "The old key components need to be taken to a locksmith or dealership to be reprogrammed and cut to fit the new key head.", "prevalence": "7"}]\n```'

### Run

In [9]:
from utils.postprocessing import *

In [10]:
df['my_category'] = 1
num_workers = 1

In [11]:
# inputs = [(qqsum_output_path + "/post_processed_cache",
inputs = [(qqsum_output_path + "/post_processed_cache/rd",
           domain,
           df[df['my_category'] == domain].reset_index(drop=True)
           )
          for domain in df['my_category'].unique()]

In [12]:
start_time = time.time()
with Pool(num_workers) as processor:
    data = processor.starmap(prompted_claim_split_generation, inputs)
print("TIME ELAPSED", time.time() - start_time)

1 :  Loaded saved file. Done
TIME ELAPSED 0.2147808074951172


In [13]:
processed_df = pd.concat(data)
processed_df = processed_df[processed_df['comment_clusters'].str.len() > 0]
processed_df = processed_df[processed_df['final_summary'].str.len() > 0]

In [14]:
processed_df['claim_split_predicted'] = processed_df['claim_split_predicted'].apply(lambda x: re.sub(r"\n+ *", "", x.replace("json", "").replace("`", "")))
mask = processed_df['claim_split_predicted'].apply(lambda x: len(re.findall(r"(: *)\'((?:[^':]*\'+[^':,]*)+)\'( *)", x, re.DOTALL)) > 0)
processed_df.loc[mask, 'claim_split_predicted'] = processed_df.loc[mask, 'claim_split_predicted'].apply(
    lambda x: re.sub(r"(: *)\'((?:[^':]*\'+[^':]*)+)\'( *,)", r'\1"""\2"""\3', x))
processed_df['claim_split_predicted'] = processed_df['claim_split_predicted'].apply(extract_claims)

In [15]:
processed_df = processed_df.reset_index(drop=True)

In [16]:
processed_df = processed_df.apply(match_claim_with_cluster, axis=1)

In [17]:
processed_df = processed_df.rename(columns={'key_points': 'key_point_given'})

In [18]:
claim_split_predicted = pd.json_normalize(
    processed_df.to_dict(orient='records'), 
    "matching_comment_clusters", ["category", "asin", "id", "query", "passages", 'passages_len']
)
claim_split_predicted

,comments,cluster_size,key_point,prevalence,category,asin,id,query,passages,passages_len
0,[[the plastic case of my old keys was broken a...,13,the key head and the plastic case are integrat...,13,Automotive,B006FDD74W,3609,broke key head --can I just get new key head a...,[[the plastic case of my old keys was broken a...,26
1,[Push the transponder into the new key head. ...,3,pushing the transponder into the new key head ...,3,Automotive,B006FDD74W,3609,broke key head --can I just get new key head a...,[[the plastic case of my old keys was broken a...,26
2,"[I got this to replace a broken key remote., B...",3,replacing the broken shells on old keys,3,Automotive,B006FDD74W,3609,broke key head --can I just get new key head a...,[[the plastic case of my old keys was broken a...,26
3,[As it is I used the good half with the old ke...,4,"the need to replace the metal area of the key,...",4,Automotive,B006FDD74W,3609,broke key head --can I just get new key head a...,[[the plastic case of my old keys was broken a...,26
4,"[In order to use this key, you will need two w...",28,you need two original working keys to program ...,28,Automotive,B005OZNZD0,2799,I have a base model 2007 F150 XL V6. I would l...,"[In order to use this key, you will need two w...",39
...,...,...,...,...,...,...,...,...,...,...
192,[inFamous Second Son however doesn't keep you ...,24,Infamous Second Son offers a bigger game with ...,24,Video_Games,B00FYIXMHQ,1837,I've never played Infamous 1 - Should you find...,[inFamous Second Son however doesn't keep you ...,34
193,"[In fact, before playing I was very skeptical ...",7,Infamous Second Son has impressive graphics an...,7,Video_Games,B00FYIXMHQ,1837,I've never played Infamous 1 - Should you find...,[inFamous Second Son however doesn't keep you ...,34
194,"[In terms of controls, the game allows you wit...",13,"The game allows the use of Wii remote only, re...",13,Video_Games,B00269DXXO,2189,do you have to use the remote and nunchuck? or...,"[In terms of controls, the game allows you wit...",13
195,"[so mod install will not work. , The thing is...",4,the mod installer does not work for some mods,4,Video_Games,B0049B2FP4,2484,How do you fix the mod installer and download ...,"[so mod install will not work. , The thing is...",4


In [19]:
claim_split_predicted = claim_split_predicted.groupby(['asin', 'category', 'id', 'query']).filter(lambda grp: len(grp) > 1)

In [20]:
def other_kps_same_query(grp):
    other_kps_list = []
    for i, row in grp.iterrows():
        my_kp = row['key_point']
        other_kps = [other_kp for other_kp in grp['key_point'] if other_kp != my_kp]
        other_kps_list += [other_kps]
    
    grp['key_point_given'] = other_kps_list
    
    return grp
    
claim_split_predicted = claim_split_predicted.groupby(['asin', 'category', 'id', 'query'], sort=False).apply(other_kps_same_query).reset_index(drop=True)

In [21]:
claim_split_predicted[['key_point', 'key_point_given']].iloc[0]['key_point']

'the key head and the plastic case are integrated, requiring a visit to a locksmith or dealership to get it cut and the old key embedded in it'

In [22]:
claim_split_predicted[['key_point', 'key_point_given']].iloc[0]['key_point_given']

['pushing the transponder into the new key head and securing it in place',
 'replacing the broken shells on old keys',
 'the need to replace the metal area of the key, but were unable to do so with this product']

In [23]:
merged_df = claim_split_predicted.explode(['key_point_given'])
merged_df = merged_df.reset_index(drop=True)

In [24]:
merged_df

,comments,cluster_size,key_point,prevalence,category,asin,id,query,passages,passages_len,key_point_given
0,[[the plastic case of my old keys was broken a...,13,the key head and the plastic case are integrat...,13,Automotive,B006FDD74W,3609,broke key head --can I just get new key head a...,[[the plastic case of my old keys was broken a...,26,pushing the transponder into the new key head ...
1,[[the plastic case of my old keys was broken a...,13,the key head and the plastic case are integrat...,13,Automotive,B006FDD74W,3609,broke key head --can I just get new key head a...,[[the plastic case of my old keys was broken a...,26,replacing the broken shells on old keys
2,[[the plastic case of my old keys was broken a...,13,the key head and the plastic case are integrat...,13,Automotive,B006FDD74W,3609,broke key head --can I just get new key head a...,[[the plastic case of my old keys was broken a...,26,"the need to replace the metal area of the key,..."
3,[Push the transponder into the new key head. ...,3,pushing the transponder into the new key head ...,3,Automotive,B006FDD74W,3609,broke key head --can I just get new key head a...,[[the plastic case of my old keys was broken a...,26,the key head and the plastic case are integrat...
4,[Push the transponder into the new key head. ...,3,pushing the transponder into the new key head ...,3,Automotive,B006FDD74W,3609,broke key head --can I just get new key head a...,[[the plastic case of my old keys was broken a...,26,replacing the broken shells on old keys
...,...,...,...,...,...,...,...,...,...,...,...
263,[Atleast on Xbox 360 you do.- mic isn't detach...,4,the microphone is not detachable and if it bre...,4,Video_Games,B00EIM3BV6,1004,I have plugged in the cord from the headset to...,"[The headset would not turn off, had a frequen...",44,the microphone works intermittently and requir...
264,[but these definitely can be used for everythi...,8,"these headsets can be used for everything, inc...",8,Video_Games,B00EIM3BV6,2505,I like to use Skype on my computer while using...,[but these definitely can be used for everythi...,40,the headset's sound quality for gaming and eve...
265,[It is completely wireless except if you are a...,30,the headset's sound quality for gaming and eve...,30,Video_Games,B00EIM3BV6,2505,I like to use Skype on my computer while using...,[but these definitely can be used for everythi...,40,"these headsets can be used for everything, inc..."
266,[inFamous Second Son however doesn't keep you ...,24,Infamous Second Son offers a bigger game with ...,24,Video_Games,B00FYIXMHQ,1837,I've never played Infamous 1 - Should you find...,[inFamous Second Son however doesn't keep you ...,34,Infamous Second Son has impressive graphics an...


# Evaluation

In [25]:
from softF1 import *

## G-Eval

In [26]:
prompt = get_prompt("experiment_g_eval_redundancy")

### Trial

In [28]:
row = merged_df.iloc[0]
output_list = g_eval(prompt, row['key_point_given'], row['key_point'])
statistics.mean(process_g_eval(output_list))

1

### Run

In [29]:
num_workers = 1
merged_df['my_category'] = 1

In [30]:
inputs = [(qqsum_output_path + "/g_eval_cache/kp_redundancy",
           domain,
           merged_df[merged_df['my_category'] == domain].reset_index(drop=True)
           )
          for domain in merged_df['my_category'].unique()]

In [31]:
start_time = time.time()
with Pool(num_workers) as processor:
    data = processor.starmap(prompted_g_eval_kp, inputs)
print("TIME ELAPSED", time.time() - start_time)

1 :  Loaded saved file. Done
TIME ELAPSED 0.07877635955810547


In [32]:
g_eval_df = pd.concat(data)

In [33]:
g_eval_df['g_eval_scores'] = g_eval_df['g_eval_scores'].apply(process_g_eval)

In [34]:
import statistics
g_eval_df['g_eval_relevancy'] = g_eval_df['g_eval_scores'].apply(lambda x: statistics.mean(x))

In [35]:
g_eval_df['g_eval_relevancy']

0      2.200000
1      2.714286
2      3.083333
3      1.727273
4      1.777778
         ...   
263    2.300000
264    2.400000
265    2.500000
266    2.411765
267    2.000000
Name: g_eval_relevancy, Length: 268, dtype: float64

In [36]:
softp_data = g_eval_df.groupby(['category', 'asin', 'id', 'query', 'key_point'])['g_eval_relevancy'].max().reset_index()

In [37]:
P_average = softp_data['g_eval_relevancy'].mean()
P_average

2.4320099258645236

In [38]:
P_average = softp_data['g_eval_relevancy'].mean()
P_average = (P_average - 1) / 4
P_average

0.3580024814661309

## BERTScore

In [39]:
softp_data = merged_df.groupby(['category', 'asin', 'id', 'query', 'key_point'])\
    .apply(lambda grp: "".join([cand + "=" for cand in (grp['key_point_given'].tolist())])).reset_index(name='multi_cands')
softp_data

,category,asin,id,query,key_point,multi_cands
0,Automotive,B000VUC51G,1012,Customers have claimed that Dee Zee has 2 very...,Dee Zee sells two different bed mats under one...,"The quality, fit, and price of the received be..."
1,Automotive,B000VUC51G,1012,Customers have claimed that Dee Zee has 2 very...,"The quality, fit, and price of the received be...",Dee Zee sells two different bed mats under one...
2,Automotive,B005OZNZD0,2799,I have a base model 2007 F150 XL V6. I would l...,"if you don't have two keys already, you will h...",you need two original working keys to program ...
3,Automotive,B005OZNZD0,2799,I have a base model 2007 F150 XL V6. I would l...,you need two original working keys to program ...,"if you don't have two keys already, you will h..."
4,Automotive,B006FDD74W,3609,broke key head --can I just get new key head a...,pushing the transponder into the new key head ...,the key head and the plastic case are integrat...
...,...,...,...,...,...,...
138,Video_Games,B00EIM3BV6,1004,I have plugged in the cord from the headset to...,the microphone works intermittently and requir...,the headset does not turn on or has buttons th...
139,Video_Games,B00EIM3BV6,2505,I like to use Skype on my computer while using...,the headset's sound quality for gaming and eve...,"these headsets can be used for everything, inc..."
140,Video_Games,B00EIM3BV6,2505,I like to use Skype on my computer while using...,"these headsets can be used for everything, inc...",the headset's sound quality for gaming and eve...
141,Video_Games,B00FYIXMHQ,1837,I've never played Infamous 1 - Should you find...,Infamous Second Son has impressive graphics an...,Infamous Second Son offers a bigger game with ...


In [40]:
cands, refs= preprocess_text(softp_data, metrics = "softPrecision")

P, R, F = bert_scorer.score(cands, refs)
P_average = P.mean()

In [41]:
P_average

tensor(0.3727)

## BARTScore

In [42]:
softp_data = merged_df.groupby(['category', 'asin', 'id', 'query', 'key_point'])\
    .apply(lambda grp: "".join([cand + "=" for cand in (grp['key_point_given'].tolist())])).reset_index(name='multi_cands')
softp_data

,category,asin,id,query,key_point,multi_cands
0,Automotive,B000VUC51G,1012,Customers have claimed that Dee Zee has 2 very...,Dee Zee sells two different bed mats under one...,"The quality, fit, and price of the received be..."
1,Automotive,B000VUC51G,1012,Customers have claimed that Dee Zee has 2 very...,"The quality, fit, and price of the received be...",Dee Zee sells two different bed mats under one...
2,Automotive,B005OZNZD0,2799,I have a base model 2007 F150 XL V6. I would l...,"if you don't have two keys already, you will h...",you need two original working keys to program ...
3,Automotive,B005OZNZD0,2799,I have a base model 2007 F150 XL V6. I would l...,you need two original working keys to program ...,"if you don't have two keys already, you will h..."
4,Automotive,B006FDD74W,3609,broke key head --can I just get new key head a...,pushing the transponder into the new key head ...,the key head and the plastic case are integrat...
...,...,...,...,...,...,...
138,Video_Games,B00EIM3BV6,1004,I have plugged in the cord from the headset to...,the microphone works intermittently and requir...,the headset does not turn on or has buttons th...
139,Video_Games,B00EIM3BV6,2505,I like to use Skype on my computer while using...,the headset's sound quality for gaming and eve...,"these headsets can be used for everything, inc..."
140,Video_Games,B00EIM3BV6,2505,I like to use Skype on my computer while using...,"these headsets can be used for everything, inc...",the headset's sound quality for gaming and eve...
141,Video_Games,B00FYIXMHQ,1837,I've never played Infamous 1 - Should you find...,Infamous Second Son has impressive graphics an...,Infamous Second Son offers a bigger game with ...


In [43]:
cands, refs= preprocess_text(softp_data, metrics = "softPrecision")

#BART score cannot be processed for different numbers of reference sentences, so
#check for the maximum number of reference sentences and match the size.
#We fill in the None for the missing sentences because we only pick one of the 
#maximum values and not the average, so there is no impact on performance

refs = balance_ref_num(refs)        

# generation scores from the first list of texts to the second list of texts.
P = bart_scorer.multi_ref_score(cands, refs, agg="max", batch_size=4) # agg means aggregation, can be mean or max

#mapping the score to (0,1]
P_average = math.tanh(math.exp((mean(P))/2+1.3))
#P_average = math.tanh(mean(P)) + 1
#P_average = math.exp(mean(P))

In [44]:
P_average

0.5046201000985896

## BLEURTScore

In [45]:
softp_data = merged_df.sort_values(by=['category', 'asin', 'id', 'query', 'key_point'])
df_compare_precision = softp_data[['key_point', 'key_point_given']].rename(columns={'key_point': 'candidate', 'key_point_given': 'reference'})
df_compare_precision

,candidate,reference
14,Dee Zee sells two different bed mats under one...,"The quality, fit, and price of the received be..."
15,"The quality, fit, and price of the received be...",Dee Zee sells two different bed mats under one...
13,"if you don't have two keys already, you will h...",you need two original working keys to program ...
12,you need two original working keys to program ...,"if you don't have two keys already, you will h..."
3,pushing the transponder into the new key head ...,the key head and the plastic case are integrat...
...,...,...
261,the microphone works intermittently and requir...,the microphone is not detachable and if it bre...
265,the headset's sound quality for gaming and eve...,"these headsets can be used for everything, inc..."
264,"these headsets can be used for everything, inc...",the headset's sound quality for gaming and eve...
267,Infamous Second Son has impressive graphics an...,Infamous Second Son offers a bigger game with ...


In [46]:
candidates = df_compare_precision['candidate']
references = df_compare_precision['reference']

In [47]:
import tensorflow as tf

In [48]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


2025-02-22 03:21:07.020570: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-22 03:21:07.125047: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-22 03:21:07.125343: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [49]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8396573695873806923
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 19425918976
locality {
  bus_id: 1
  links {
  }
}
incarnation: 975311914435770980
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9"
xla_global_id: 416903419
]


2025-02-22 03:21:07.147825: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-22 03:21:07.148995: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-22 03:21:07.149266: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-22 03:21:07.150306: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-22 03:21:07.150967: I external/local_xla/xla/stream_executor

In [50]:
softp_data = merged_df.sort_values(by=['category', 'asin', 'id', 'query', 'key_point'])
df_compare_precision = softp_data[['key_point', 'key_point_given']].rename(columns={'key_point': 'candidate', 'key_point_given': 'reference'})
df_compare_precision

,candidate,reference
14,Dee Zee sells two different bed mats under one...,"The quality, fit, and price of the received be..."
15,"The quality, fit, and price of the received be...",Dee Zee sells two different bed mats under one...
13,"if you don't have two keys already, you will h...",you need two original working keys to program ...
12,you need two original working keys to program ...,"if you don't have two keys already, you will h..."
3,pushing the transponder into the new key head ...,the key head and the plastic case are integrat...
...,...,...
261,the microphone works intermittently and requir...,the microphone is not detachable and if it bre...
265,the headset's sound quality for gaming and eve...,"these headsets can be used for everything, inc..."
264,"these headsets can be used for everything, inc...",the headset's sound quality for gaming and eve...
267,Infamous Second Son has impressive graphics an...,Infamous Second Son offers a bigger game with ...


In [51]:
candidates = df_compare_precision['candidate']
references = df_compare_precision['reference']

In [52]:
import tensorflow as tf

In [53]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [54]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16437338414538392554
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 19425918976
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7868647629857499368
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9"
xla_global_id: 416903419
]


2025-02-22 03:21:09.585987: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-22 03:21:09.586304: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-22 03:21:09.586536: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-22 03:21:09.587019: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-22 03:21:09.587028: I tensorflow/core/common_runtime/gpu/gpu

In [55]:
result = calculatingScore(references, candidates)
df_compare_precision["BLEURT Score"] = result

#After calculating the semantic quality of all candidates and reference pairs, the one with the highest score is selected as the correct pair.
df_bestkp_pair_precision = df_compare_precision.loc[df_compare_precision.groupby(["candidate"])["BLEURT Score"].idxmax()]
#take average of all best scores as the soft precision score.
P_average = df_bestkp_pair_precision["BLEURT Score"].mean()

INFO:tensorflow:Reading checkpoint /mnt/d/Desktop/PHD READING/QQSUMM/evaluation/KeyPoint-Analysis/KPG/BLEURT-20.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint BLEURT-20
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:BLEURT-20
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... max_seq_length:512
INFO:tensorflow:... vocab_file:None
INFO:tensorflow:... do_lower_case:None
INFO:tensorflow:... sp_model:sent_piece
INFO:tensorflow:... dynamic_seq_length:True
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating SentencePiece tokenizer.
INFO:tensorflow:Creating SentencePiece tokenizer.
INFO:tensorflow:Will load model: /mnt/d/Desktop/PHD READING/QQSUMM/evaluation/KeyPoint-Analysis/KPG/BLEURT-20/sent_piece.model.
INFO:tensorflow:SentencePiece tokenizer created.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.


2025-02-22 03:21:11.814894: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-22 03:21:11.815346: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-22 03:21:11.815682: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-22 03:21:11.816278: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-22 03:21:11.816289: I tensorflow/core/common_runtime/gpu/gpu

INFO:tensorflow:BLEURT initialized.


INFO:tensorflow:BLEURT initialized.


In [56]:
P_average

0.4891105825667615